# Business Requirement:

So this week we are looking at a problem in ***cybersecurity.*** We're dealing with a big company here that has a lot of firewall interaction. Now what they're asking us to do is to build a program about whether or not to accept or deny access to requests coming in.

They want us to automate it because, currently, their current system is taking up a lot of manpower and a lot of resources. So they have all the ***historical data** from before of how they've chosen to accept and deny these requests. So the model needs to be able to filter the incoming requests.

We need to classify them. And then it needs to be able to auto accept or auto deny the request. We want the best, most advanced model. It needs to be accurate, and it needs to function at speed.

So you have about ***60,000 rows*** of data to play with. And in the end, whether we put this model together, we're going to need a report on the level of accuracy and the level of performance so that they can make the decision on whether or not this can be implemented. 

***Build firewall classification model to classify whether or not to accept or deny access to requests coming in***